# 기존 Dataframe에 도로명 좌표 컬럼 추가

In [2]:
import pandas as pd

In [3]:
# Geopy 기반 좌표 변환 결과 데이터프레임과 API 기반 좌표 변환 결과 데이터프레임 병합
df_Geopy = pd.read_csv('II-1. Only_Geopy_result.csv')
df_Geopy.shape

(10842, 3)

In [4]:
df_API = pd.read_csv('II-1. df_roadCoord_API.csv')
df_API.shape

(637, 3)

In [5]:
# df_Geopy의 address컬럼과 df_API의 address컬럼을 기준으로 병합
# 즉, df_API에 있는 도로명 주소와 좌표 정보를 기준으로 df_Geopy에 좌표 정보를 수정 및 추가
# 즉, df_API정보를 바탕으로 df_Geopy정보를 업데이트 합니다.
merged_left_df = df_Geopy.merge(df_API, on='address', how='left')
merged_left_df.shape

(10842, 5)

In [7]:
import numpy as np

df = merged_left_df  # merged_left_df가 이미 존재한다고 가정

for coord in ['lat', 'lon']:
    x_col, y_col = f'{coord}_x', f'{coord}_y'
    if x_col in df.columns and y_col in df.columns:
        # 숫자형 변환(문자 '0.0' 등 처리)
        df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
        df[y_col] = pd.to_numeric(df[y_col], errors='coerce')

        # x가 0.0 또는 NaN이고, y가 값이 있을 때만 대체
        mask = (df[x_col].isna() | (df[x_col] == 0.0)) & df[y_col].notna()
        replaced = int(mask.sum())
        df.loc[mask, x_col] = df.loc[mask, y_col]
        print(f'{x_col}: {replaced}건 대체')
    else:
        print(f'{x_col} 또는 {y_col} 컬럼이 없습니다.')

# 후속 사용을 위해 결과를 merged_df에 보관
merged_df = df.copy()

lat_x: 2115건 대체
lon_x: 2115건 대체


In [9]:
# lat_y, lon_y 컬럼 제거
merged_df = merged_df.drop(columns=['lat_y', 'lon_y'])

In [11]:
# 이름 변경
merged_df = merged_df.rename(columns={'lat_x': 'lat', 'lon_x' : 'lon'})

In [ ]:
# csv로 임시저장
# merged_df.to_csv('II-3. Merged_Geopy_API_result.csv', index=False, encoding='utf-8-sig')

In [20]:
# A 데이터프레임의 '도로명'컬럼의 값을 기준으로 B 데이터프레임의 'address'컬럼과 매칭하여
# A 데이터프레임에 B 데이터프레임의 '위도', '경도'컬럼 값을 추가.
A_df = pd.read_csv('I-3. merged_df_processed.csv')  # A 데이터프레임 로드
# A_df에 비어있는 lat, lon 컬럼 추가
A_df[['lat', 'lon']] = np.nan
A_df.head()


,시군구,번지,주택유형,도로조건,연면적(㎡),대지면적(㎡),계약년월,계약일,거래금액(만원),건축년도,도로명,비고,lat,lon
0,경상남도 창원성산구 가음동,3*,다가구,12m미만,231.78,255.2,201508,25,"53,000",1997.0,가양로52번길,2. 성산구_단독다가구(매매),NaN,NaN
1,경상남도 창원성산구 가음동,4*,다가구,12m미만,236.05,257.4,201606,27,"53,000",1999.0,가음로92번길,2. 성산구_단독다가구(매매),NaN,NaN
2,경상남도 창원성산구 가음동,*,다가구,12m미만,381.05,268.7,201604,14,"70,000",2013.0,가음정로23번길,2. 성산구_단독다가구(매매),NaN,NaN
3,경상남도 창원성산구 가음동,*,다가구,12m미만,265.88,269.7,201605,16,"65,000",2016.0,가음정로11번길,2. 성산구_단독다가구(매매),NaN,NaN
4,경상남도 창원성산구 가음동,4*,단독,12m미만,350.01,248.0,201509,24,"60,000",1997.0,가양로34번길,2. 성산구_단독다가구(매매),NaN,NaN


In [23]:
# A_df의 '도로명' 컬럼의 값과 merged_df의 'address'컬럼의 값을 매칭하여 lat, lon 값 추가.
# merged_df에 같은 address가 여러개 있을 수 있으므로 index가 유일하도록 처리한 뒤 매핑을 만듭니다.
address_to_coords = merged_df.drop_duplicates(subset='address').set_index('address')[['lat', 'lon']].to_dict(orient='index')

# map을 사용하여 벡터화된 방식으로 할당 (루프보다 빠르고 간결)
A_df['lat'] = A_df['도로명'].map(lambda a: address_to_coords.get(a, {}).get('lat') if pd.notna(a) else np.nan)
A_df['lon'] = A_df['도로명'].map(lambda a: address_to_coords.get(a, {}).get('lon') if pd.notna(a) else np.nan)

A_df.head()

,시군구,번지,주택유형,도로조건,연면적(㎡),대지면적(㎡),계약년월,계약일,거래금액(만원),건축년도,도로명,비고,lat,lon
0,경상남도 창원성산구 가음동,3*,다가구,12m미만,231.78,255.2,201508,25,"53,000",1997.0,가양로52번길,2. 성산구_단독다가구(매매),35.205613,128.701453
1,경상남도 창원성산구 가음동,4*,다가구,12m미만,236.05,257.4,201606,27,"53,000",1999.0,가음로92번길,2. 성산구_단독다가구(매매),35.204905,128.701755
2,경상남도 창원성산구 가음동,*,다가구,12m미만,381.05,268.7,201604,14,"70,000",2013.0,가음정로23번길,2. 성산구_단독다가구(매매),35.210464,128.688074
3,경상남도 창원성산구 가음동,*,다가구,12m미만,265.88,269.7,201605,16,"65,000",2016.0,가음정로11번길,2. 성산구_단독다가구(매매),35.211962,128.685748
4,경상남도 창원성산구 가음동,4*,단독,12m미만,350.01,248.0,201509,24,"60,000",1997.0,가양로34번길,2. 성산구_단독다가구(매매),35.204416,128.700351


In [24]:
# csv로 저장
A_df.to_csv('II-3. Final_A_df_with_Coords.csv', index=False, encoding='utf-8-sig')